In [2]:
# Changing basis

In [1]:
import pygsti
from pygsti.extras.interpygate import *

In [4]:
from pygsti.extras.interpygate import do_process_tomography, vec, unvec

In [8]:
from pygsti.tools.basistools import change_basis

In [10]:
from mpi4py import MPI
_comm = MPI.COMM_WORLD
_rank = _comm.Get_rank()
_size = _comm.Get_size()

In [16]:
import numpy as _np
from scipy.linalg import expm as _expm

In [ ]:
class ProcessFunction(object):
    def __init__(self):
        import numpy as _np
        import scipy as _sp

        self.Hx = _np.array([[0, 0, 0, 0],
                            [0, 0, 0, 0],
                            [0, 0, 0, -1],
                            [0, 0, 1, 0]], dtype='float')
        self.Hy = _np.array([[0, 0, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 0, 0],
                            [0, -1, 0, 0]], dtype='float')
        self.Hz = _np.array([[0, 0, 0, 0],
                            [0, 0, -1, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 0]], dtype='float')

        self.dephasing_generator = _np.diag([0, -1, -1, 0])
        self.decoherence_generator = _np.diag([0, -1, -1, -1])

    def advance(self, state, v = None, times = None):
        state = _np.array(state, dtype='complex')
        if times is None:
            t, omega, phase, detuning, dephasing, decoherence = v
            times = [t]
        else:
            omega, phase, detuning, dephasing, decoherence = v

        H = (omega * _np.cos(phase) * self.Hx + omega * _np.sin(phase) * self.Hy + detuning * self.Hz)
        L = dephasing * self.dephasing_generator + decoherence * self.decoherence_generator

        processes = [change_basis(_expm((H + L) * t),
                                                          'pp', 'col') for t in times]
        states = [unvec(_np.dot(process, vec(_np.outer(state, state.conj())))) for process in processes]

        return states

    def __call__(self, v, times=None, comm=_comm):
        print(f'Calling process tomography as {comm.Get_rank()} of {comm.Get_size()} on {comm.Get_name()}.')
        processes = do_process_tomography(self.advance, opt_args={'v':v, 'times':times},
                                          n_qubits = 1, basis='pp', time_dependent=True, comm=comm)

        return processes

gy = PhysicalProcess(mpi_workers_per_process=1, basis='col')
gy.set_process_function(ProcessFunction(), mpi_enabled=True)
target = change_basis(_np.array([[1,0,0,0],
                                 [0,1,0,0],
                                 [0,0,0,-1],
                                 [0,0,1,0]], dtype='complex'), 'pp', 'pp')
gy.set_target(target)

# # # Evaluate one time point per run
# # gy.set_parameter_range([[_np.pi/2-.5, _np.pi/2+.5],[0.9,1.1],[-.1,.1],[-.1,.1],[0,0.1],[0,0.1]])
# # gy.set_interpolation_order([3,3,3,3,3,3])
#
# Evaluate many time points per run
# Notice that the number of parameters listed below is 1 fewer than in the previous example
gy.set_parameter_range([[0.9, 1.1], [-.1, .1], [-.1, .1], [0, 0.1], [0, 0.1]])
gy.set_interpolation_order([3,3,3,3,3])
gy.set_times(_np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10))

gy.interpolate()

Group 0 processing 243 points on 1 processors.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 

Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography a

# Testing interpygate feature in pyGSTi

In [5]:
""" Demonstrate the process tomography function with (potentially) time-dependent outputs. """
sigI = _np.array([[1, 0], [0, 1]], dtype='complex')
sigX = _np.array([[0, 1], [1, 0]], dtype='complex')
sigY = _np.array([[0, -1.j], [1.j, 0]], dtype='complex')
sigZ = _np.array([[1, 0], [0, -1]], dtype='complex')
theta = .32723
u = _np.cos(theta) * sigI + 1.j * _np.sin(theta) * sigX
v = _np.sin(theta) * sigI - 1.j * _np.cos(theta) * sigX

U = _np.kron(u, v)
n_qubits = 2
# U = u
# n_qubits = 1
test_process = _np.kron(U.conj().T, U)


def single_time_test_function(pure_state, test_process=test_process):
    rho = vec(_np.outer(pure_state, pure_state.conj()))
    return unvec(_np.dot(test_process, rho))


def multi_time_test_function(pure_state, test_process=test_process):
    rho = vec(_np.outer(pure_state, pure_state.conj()))
    return [unvec(_np.dot(test_process, rho)), unvec(_np.dot(_np.linalg.matrix_power(test_process,2), rho))]


process_matrix = do_process_tomography(single_time_test_function, n_qubits=2, verbose=False)
if _rank == 0:
    test_process_pp = change_basis(test_process, 'col', 'pp')
    print("\nSingle-time test result should be True:")
    print(_np.isclose(process_matrix, test_process_pp).all())

process_matrices = do_process_tomography(multi_time_test_function, n_qubits=2, verbose=False, time_dependent=True)
if _rank==0:
    test_process = change_basis(test_process, 'col', 'pp')
    print("\nMulti-time test result should be [True, False]:")
    print([_np.isclose(x, test_process).all() for x in process_matrices])


Single-time test result should be True:
True

Multi-time test result should be [True, False]:
[True, False]
